In [1]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd
from selenium.webdriver.common.by import By
import psycopg2
import subprocess
import yaml
import sqlalchemy as sa
import time
import re

from selenium.webdriver.common.action_chains import ActionChains

In [2]:
with open(r'\Users\jasre\Code\ws_config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

# Running To Do
- Parse follow ups for JP versions of characters (manual...)
- Add notes for characters:
  - Deuce -- Her Sforzando buff isn't included in the BT aura and will have do be added in the visual
- Set up marts
- Set up Streamlit

In [3]:
cs = CompendiumScraper('\\Users\\jasre\\Code\\ws_config.yml')

2023-09-04 16:19:21,144 - web_scraper.web_scraper - ====================================================
2023-09-04 16:19:21,144 - web_scraper.web_scraper - Retrieved all main character links. Generating remaining links for each character.
2023-09-04 16:19:21,144 - web_scraper.web_scraper - ====================================================
2023-09-04 16:19:21,187 - web_scraper.web_scraper - self.character_dict_omnibus entry for JIHLNABAAT was successful
2023-09-04 16:19:21,212 - web_scraper.web_scraper - self.character_dict_omnibus entry for WEISS was successful
2023-09-04 16:19:21,237 - web_scraper.web_scraper - self.character_dict_omnibus entry for ASTOS was successful
2023-09-04 16:19:21,307 - web_scraper.web_scraper - self.character_dict_omnibus entry for IRIS was successful
2023-09-04 16:19:21,330 - web_scraper.web_scraper - self.character_dict_omnibus entry for REKS was successful
2023-09-04 16:19:21,375 - web_scraper.web_scraper - self.character_dict_omnibus entry for QUINA w

In [4]:
char_name = 'deuce'

bt_df = cs.retrieve_hp_caps_from_bt(char_name=char_name, JP=True, return_output=True)

bt_df

2023-09-04 16:19:24,580 - web_scraper.web_scraper - ====================================================
2023-09-04 16:19:24,581 - web_scraper.web_scraper - Retrieving BT info for DEUCE.
2023-09-04 16:19:24,581 - web_scraper.web_scraper - ====================================================
2023-09-04 16:19:37,549 - web_scraper.web_scraper - Standard BT stack slider div-class not found. Trying second possibility.
2023-09-04 16:19:37,557 - web_scraper.web_scraper - No stack slider found. Assuming DEUCE has a BT without stacks.
2023-09-04 16:19:37,653 - web_scraper.web_scraper - Retrieved BT info for DEUCE.


,char_name,bt_personal_hp_dmg_cap_up,bt_party_hp_dmg_cap_up,enemy_count_apply_list,game_version
0,deuce,0,30,"[1, 2, 3]",JP


In [4]:
ability_df = cs.generate_ability_df(char_name=char_name, JP=False)

ability_df

2023-09-04 11:27:43,382 - web_scraper.web_scraper - ====================================================
2023-09-04 11:27:43,383 - web_scraper.web_scraper - Retrieving BT info for RUFUS.
2023-09-04 11:27:43,383 - web_scraper.web_scraper - ====================================================
2023-09-04 11:27:56,365 - web_scraper.web_scraper - Standard BT stack slider div-class not found. Trying second possibility.
2023-09-04 11:27:56,370 - web_scraper.web_scraper - No stack slider found. Assuming rufus has a BT without stacks.
2023-09-04 11:27:56,432 - web_scraper.web_scraper - Retrieved BT info for RUFUS.


,char_name,bt_personal_hp_dmg_cap_up,bt_party_hp_dmg_cap_up,enemy_count_apply_list,game_version
0,rufus,0,30,"[1, 2, 3]",JP


In [6]:
ha_df = cs.retrieve_ha_hp_dmg_cap_up(char_name=char_name, JP=True, return_output=True)

ha_df

2023-09-04 11:28:55,132 - web_scraper.web_scraper - ====================================================
2023-09-04 11:28:55,133 - web_scraper.web_scraper - Retrieving High Armor info for RUFUS.
2023-09-04 11:28:55,133 - web_scraper.web_scraper - ====================================================
2023-09-04 11:29:05,985 - web_scraper.web_scraper - High armor info parsed for RUFUS.


,char_name,personal_hp_dmg_cap_up,party_ha_hp_dmg_cap_up,game_version
0,rufus,45,0,JP


# Following can be used to manually upload data to Postgres if the web scraper fails to do so. 
## DO NOT DELETE!

In [ ]:
# SQL alchemy stuff

engine_url = sa.URL.create(
        "postgresql",
        username=config['pg_user'],
        password=config['pg_pass'],
        host=config['pg_host'],
        database=config['pg_db']
    )
    
engine = sa.create_engine(engine_url)

bt_effects = pd.read_csv(config['datasets_dir'] + 'raw_bt_effects.csv')
abilities = pd.read_csv(config['datasets_dir'] + 'raw_abilities.csv')
ha_caps = pd.read_csv(config['datasets_dir'] + 'raw_high_armor_caps.csv')

with engine.begin() as conn:
    abilities.to_sql('raw_abilities', con=conn, index=False, if_exists='append')
    bt_effects.to_sql('raw_bt_effects', con=conn, index=False, if_exists='append')
    ha_caps.to_sql('raw_high_armor_caps', con=conn, index=False, if_exists='append')